# model

In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import explained_variance_score
from lightgbm import LGBMRegressor, plot_importance
from sklearn.metrics import roc_auc_score

In [2]:
# knob 종류 및 바운드
[305, 722, 5, 12, 9163, 'no', 'yes']
knobs_rdb = {
    
    "hash-max-ziplist-entries":     (256, 751), 
    "hash-max-ziplist-value": (16, 257),
    "activerehashing": ('yes', 'no'),  ##yes
    "hz": (1, 41),
    "dynamic-hz": ('yes', 'no'),

    "save0_0": (700,1100),
    "save0_1": (1,9),
    "save1_0": (100,500),
    "save1_1": (10,100),
    "save2_0": (30,90),
    "save2_1": (8000,12000),

    "rdbcompression": ('yes', 'no'), ##yes
    "rdbchecksum": ('yes', 'no'),    ##yes
    "rdb-save-incremental-fsync": ('yes', 'no'),   ## yes

    "activedefrag": ('yes','no'),  
    "active-defrag-threshold-lower": (1, 31),
    "active-defrag-threshold-upper": (70, 101),
    "active-defrag-cycle-min": (1, 31),
    "active-defrag-cycle-max": (70, 91),
    
    "maxmemory": (1000, 2900),   ##"volatile-lru"
    "maxmemory-policy":     ("volatile-lru", "allkeys-lru", "volatile-lfu", "allkeys-lfu", 
                             "volatile-random","allkeys-random", "volatile-ttl", "noeviction"),
    "maxmemory-samples": (3, 7),
    
    "lazyfree-lazy-eviction": ('yes', 'no'),
    "lazyfree-lazy-expire": ('yes', 'no'),
    "lazyfree-lazy-server-del": ('yes', 'no')
}

knobs_list = list(knobs_rdb.keys())
knobs_bound = list(knobs_rdb.values())

## 데이터 전처리
* 이상치 파악
* test, train 데이터 잘 나뉘어졌는지 확인

In [3]:
# config(knob) data
# config_df = pd.read_csv("./data/result_config0.csv", sep=',')
config_df = pd.read_csv("./data/result_config5.csv", sep=',')

# external(throughput) data
# external_df = pd.read_csv("./data/external_metrics0.csv", sep=',')
external_df = pd.read_csv("./data/external_metrics5.csv", sep=',')

In [4]:
# config(knog), external(throughput) 데이터 하나의 프레임으로 결합
dataset = config_df
dataset['throughput'] = external_df['Gets_KB/sec']

In [5]:
# 결측치 처리 
dataset_nan = dataset.copy()
dataset['activedefrag'].fillna(value='no', inplace=True)  # activedefrag no 채우기
dataset = dataset.fillna(value = 0)  # 전부 0으로 채움

In [6]:
dataset.head(3)

,appendonly,appendfsync,auto-aof-rewrite-percentage,auto-aof-rewrite-min-size,no-appendfsync-on-rewrite,aof-rewrite-incremental-fsync,aof-use-rdb-preamble,save0_0,save0_1,save1_0,...,maxmemory-samples,lazyfree-lazy-eviction,lazyfree-lazy-expire,lazyfree-lazy-server-del,hash-max-ziplist-entries,hash-max-ziplist-value,activerehashing,hz,dynamic-hz,throughput
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,868,4,320,...,5,yes,yes,yes,506,144,no,20,yes,45305.88
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,867,6,258,...,4,yes,no,yes,552,141,no,22,yes,44540.19
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,911,6,319,...,5,yes,no,yes,552,185,yes,20,no,45261.71


In [7]:
# maxmemory knob 단위(mb) 처리
dataset['maxmemory']  = dataset['maxmemory'].str.strip('mb')
dataset['maxmemory'] = pd.to_numeric(dataset['maxmemory'])

In [8]:
# Object 변수 categorical 로 변경 (LightGBM에 바로 넣기 위해)
category_knob = []

for knob in dataset:
    if dataset[knob].dtype == 'O':  # Object인 경우 category 형으로 변경
        dataset[knob] = dataset[knob].astype('category')
        if knob in knobs_list:
            category_knob.append(knob)

In [9]:
# 설정한 knob들만 추출
extract_knobs_list = knobs_list.copy()
extract_knobs_list.append('throughput')  # target 값 추가

dataset = dataset[extract_knobs_list]

In [10]:
dataset.head(3)

,hash-max-ziplist-entries,hash-max-ziplist-value,activerehashing,hz,dynamic-hz,save0_0,save0_1,save1_0,save1_1,save2_0,...,active-defrag-threshold-upper,active-defrag-cycle-min,active-defrag-cycle-max,maxmemory,maxmemory-policy,maxmemory-samples,lazyfree-lazy-eviction,lazyfree-lazy-expire,lazyfree-lazy-server-del,throughput
0,506,144,no,20,yes,868,4,320,58,58,...,0.0,0.0,0.0,1500,volatile-random,5,yes,yes,yes,45305.88
1,552,141,no,22,yes,867,6,258,52,67,...,0.0,0.0,0.0,1500,allkeys-lfu,4,yes,no,yes,44540.19
2,552,185,yes,20,no,911,6,319,54,65,...,0.0,0.0,0.0,2900,noeviction,5,yes,no,yes,45261.71


In [11]:
# 결측치 처리전 확인
print(dataset_nan.info())

# 결측치 처리후 확인
print(dataset.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1700 entries, 0 to 1699
Data columns (total 33 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   appendonly                     0 non-null      float64
 1   appendfsync                    0 non-null      float64
 2   auto-aof-rewrite-percentage    0 non-null      float64
 3   auto-aof-rewrite-min-size      0 non-null      float64
 4   no-appendfsync-on-rewrite      0 non-null      float64
 5   aof-rewrite-incremental-fsync  0 non-null      float64
 6   aof-use-rdb-preamble           0 non-null      float64
 7   save0_0                        1700 non-null   int64  
 8   save0_1                        1700 non-null   int64  
 9   save1_0                        1700 non-null   int64  
 10  save1_1                        1700 non-null   int64  
 11  save2_0                        1700 non-null   int64  
 12  save2_1                        1700 non-null   i

### 이상치 처리

In [12]:
# -999999와 같은 이상치 확인 -> 없음
dataset.describe()

,hash-max-ziplist-entries,hash-max-ziplist-value,hz,save0_0,save0_1,save1_0,save1_1,save2_0,save2_1,active-defrag-threshold-lower,active-defrag-threshold-upper,active-defrag-cycle-min,active-defrag-cycle-max,maxmemory,maxmemory-samples,throughput
count,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000,1700.000000
mean,503.073529,136.014118,20.331765,899.707647,5.018824,301.504118,54.734706,60.100588,10003.781176,7.648235,41.854118,7.588824,39.392353,1938.529412,4.999412,44384.915482
std,40.678152,20.991291,3.175918,33.353880,0.747528,33.671656,7.505045,5.084096,337.575574,7.950170,42.548603,7.905846,40.028266,571.372337,0.362289,2836.074574
min,365.000000,68.000000,9.000000,792.000000,2.000000,200.000000,30.000000,44.000000,8680.000000,0.000000,0.000000,0.000000,0.000000,1000.000000,4.000000,26918.970000
25%,476.000000,122.000000,18.000000,877.000000,5.000000,278.000000,50.000000,57.000000,9781.500000,0.000000,0.000000,0.000000,0.000000,1500.000000,5.000000,42970.870000
50%,503.000000,135.000000,20.000000,899.000000,5.000000,302.000000,55.000000,60.000000,9999.500000,0.000000,0.000000,0.000000,0.000000,1900.000000,5.000000,44181.360000
75%,531.000000,150.000000,22.000000,922.000000,5.000000,325.000000,60.000000,63.000000,10229.000000,15.000000,85.000000,15.000000,80.000000,2400.000000,5.000000,45033.152500
max,635.000000,204.000000,33.000000,1016.000000,7.000000,413.000000,78.000000,75.000000,11169.000000,23.000000,93.000000,23.000000,85.000000,2900.000000,6.000000,64115.000000


In [13]:
# 인덱스제거하고 돌려보자
print(dataset['throughput'][dataset['throughput'] <= 52500])
outlier_index = dataset['throughput'][dataset['throughput'] <= 52500].index
print('개수: ', len(outlier_index))

0       45305.88
1       44540.19
2       45261.71
3       45108.96
4       45277.43
          ...   
1695    44861.25
1696    45034.29
1697    45003.06
1698    44653.27
1699    44799.15
Name: throughput, Length: 1655, dtype: float64
개수:  1655


In [14]:
# # 이상치 제거
# dataset = dataset.drop(outlier_index)

In [15]:
# 이상치 제거 확인
dataset.shape

(1700, 26)

In [16]:
# 타입 확인 (카테고리)
dataset.dtypes

hash-max-ziplist-entries            int64
hash-max-ziplist-value              int64
activerehashing                  category
hz                                  int64
dynamic-hz                       category
save0_0                             int64
save0_1                             int64
save1_0                             int64
save1_1                             int64
save2_0                             int64
save2_1                             int64
rdbcompression                   category
rdbchecksum                      category
rdb-save-incremental-fsync       category
activedefrag                     category
active-defrag-threshold-lower     float64
active-defrag-threshold-upper     float64
active-defrag-cycle-min           float64
active-defrag-cycle-max           float64
maxmemory                           int64
maxmemory-policy                 category
maxmemory-samples                   int64
lazyfree-lazy-eviction           category
lazyfree-lazy-expire             c

### test, train 데이터 분리

In [17]:
# train 8, test 2
X_train_val, X_test, y_train_val, y_test = train_test_split(dataset.iloc[:,:-1]
                                                    , dataset['throughput'] 
                                                    ,test_size=0.2
                                                   , random_state = 156)

# train 6 test 2
X_train, X_val, y_train, y_val = train_test_split(X_train_val
                                                    , y_train_val 
                                                    ,test_size=0.25
                                                   , random_state = 156)


In [18]:
# X_train 확인
X_train.head()

,hash-max-ziplist-entries,hash-max-ziplist-value,activerehashing,hz,dynamic-hz,save0_0,save0_1,save1_0,save1_1,save2_0,...,active-defrag-threshold-lower,active-defrag-threshold-upper,active-defrag-cycle-min,active-defrag-cycle-max,maxmemory,maxmemory-policy,maxmemory-samples,lazyfree-lazy-eviction,lazyfree-lazy-expire,lazyfree-lazy-server-del
1645,496,127,no,19,yes,884,6,349,75,64,...,0.0,0.0,0.0,0.0,1800,volatile-random,5,yes,no,no
522,490,148,no,20,no,848,4,346,63,57,...,15.0,85.0,13.0,82.0,2900,volatile-ttl,4,yes,yes,no
445,568,147,yes,20,yes,894,6,254,59,62,...,11.0,86.0,14.0,80.0,1300,allkeys-lru,5,no,yes,no
220,580,128,yes,23,no,906,4,326,48,67,...,0.0,0.0,0.0,0.0,1900,volatile-lru,5,yes,yes,no
1611,480,136,no,18,yes,883,4,243,71,60,...,0.0,0.0,0.0,0.0,1500,volatile-ttl,5,no,no,yes


In [19]:
# # test, train 에 비슷하게 분류됬는지 확인
# y_train.hist()
# plt.title('train data')
# plt.show()

# y_test.hist()
# plt.title('test data')
# plt.show()

## LightGBM 모델 생성

In [21]:
# 모델 객체 생성, n_estimators = 400으로 설정
lgbm_wrapper = LGBMRegressor(n_estimators=1000, learning_rate=0.1)

# 검증 데이터(validation data)
evals = [(X_val, y_val)]

lgbm_wrapper.fit(X_train, y_train, early_stopping_rounds=100, eval_metric='auc',
                eval_set = evals, verbose = True, categorical_feature=category_knob)


[1]	valid_0's auc: 1	valid_0's l2: 7.24503e+06
Training until validation scores don't improve for 100 rounds
[2]	valid_0's auc: 1	valid_0's l2: 7.21582e+06
[3]	valid_0's auc: 1	valid_0's l2: 7.19e+06
[4]	valid_0's auc: 1	valid_0's l2: 7.16154e+06
[5]	valid_0's auc: 1	valid_0's l2: 7.16496e+06
[6]	valid_0's auc: 1	valid_0's l2: 7.18558e+06
[7]	valid_0's auc: 1	valid_0's l2: 7.21274e+06
[8]	valid_0's auc: 1	valid_0's l2: 7.20442e+06
[9]	valid_0's auc: 1	valid_0's l2: 7.23668e+06
[10]	valid_0's auc: 1	valid_0's l2: 7.27831e+06
[11]	valid_0's auc: 1	valid_0's l2: 7.29073e+06
[12]	valid_0's auc: 1	valid_0's l2: 7.28766e+06
[13]	valid_0's auc: 1	valid_0's l2: 7.33869e+06
[14]	valid_0's auc: 1	valid_0's l2: 7.35963e+06
[15]	valid_0's auc: 1	valid_0's l2: 7.40025e+06
[16]	valid_0's auc: 1	valid_0's l2: 7.38563e+06
[17]	valid_0's auc: 1	valid_0's l2: 7.44351e+06
[18]	valid_0's auc: 1	valid_0's l2: 7.43369e+06
[19]	valid_0's auc: 1	valid_0's l2: 7.46954e+06
[20]	valid_0's auc: 1	valid_0's l2: 7.

LGBMRegressor(n_estimators=1000)

## 성능 평가

In [22]:
# 성능 측정 함수 RMSLE(캐글에서 요구한 성능 평가 방법), MSE, RMSE
from sklearn.metrics import mean_squared_error, mean_absolute_error
# log 값 변환 시 NaN 등의 이슈로 log()가 아닌 log1p()를 이용해 RMSLE 계산 
def rmsle(y, pred):
    # log1p 변환
    log_y = np.log1p(y)
    log_pred = np.log1p(pred)
    
    # squred 계산
    squared_error = (log_y-log_pred)**2
    
    # RMSE 계산
    rmsle = np.sqrt(np.mean(squared_error))
    
    return rmsle

# 사이킷런의 mean_square_error()를 이용해 RMSE 계산
def rmse(y, pred):
    return np.sqrt(mean_squared_error(y, pred))

# MSE, RMSE, RMSLE를 모두 계산
def evaluate_regr(y, pred):
    rmsle_val = rmsle(y, pred)
    rmse_val = rmse(y, pred)
    
    # MAE는 사이킷런의 mean_absolute_error()로 계산
    mae_val = mean_absolute_error(y, pred)
    
    print('RMSLE: {0:.3f}, RMSE: {1:.3F}, MAE: {2:.3F}'.format(rmsle_val, rmse_val, mae_val))
    

In [23]:
preds = lgbm_wrapper.predict(X_test)  # 예측 결과
evaluate_regr(y_test, preds)

RMSLE: 0.052, RMSE: 2528.660, MAE: 1359.666


이상치 제거 전 RMSLE: 0.060, RMSE: 3492.223, MAE: 3032.843

이상치 제거 후 RMSLE: 0.058, RMSE: 3393.720, MAE: 2974.377

In [24]:
X_test.iloc[:1,:]

,hash-max-ziplist-entries,hash-max-ziplist-value,activerehashing,hz,dynamic-hz,save0_0,save0_1,save1_0,save1_1,save2_0,...,active-defrag-threshold-lower,active-defrag-threshold-upper,active-defrag-cycle-min,active-defrag-cycle-max,maxmemory,maxmemory-policy,maxmemory-samples,lazyfree-lazy-eviction,lazyfree-lazy-expire,lazyfree-lazy-server-del
1050,500,142,no,33,no,933,5,282,61,61,...,0.0,0.0,0.0,0.0,1200,allkeys-lfu,5,yes,yes,yes


In [25]:
# lgbm_wrapper.predict(X_test.iloc[3:4,:])

In [127]:
# 예측 prediction 시험

df_result_test = {}

for i in knobs_list:
    df_result_test[i] = ''

# bo 전체 진행 결과
df_result_test['hash-max-ziplist-entries'] = [270]  #(256, 751)
df_result_test['hash-max-ziplist-value'] = [63]  #(256, 751)
df_result_test['activerehashing'] = ['no']  #(256, 751)
df_result_test['dynamic-hz'] = ['yes']  #(256, 751)
df_result_test['hz'] = [40]  #(256, 751)
df_result_test['save0_0'] = [829]
df_result_test['save0_1'] = [1]
df_result_test['save1_0'] = [355]
df_result_test['save1_1'] = [100]
df_result_test['save2_0'] = [31]
df_result_test['save2_1'] = [8478]
df_result_test['rdbcompression'] = ['no']
df_result_test['rdbchecksum'] = ['no']
df_result_test['rdb-save-incremental-fsync'] = ['yes']
df_result_test['activedefrag'] = ['yes']

df_result_test['active-defrag-threshold-lower'] = [12]
df_result_test['active-defrag-threshold-upper'] = [86]
df_result_test['active-defrag-cycle-min'] = [10]
df_result_test['active-defrag-cycle-max'] = [80]
df_result_test['maxmemory'] = [2039]
df_result_test['maxmemory-policy'] = 'noeviction'
df_result_test['maxmemory-samples'] = [3]
df_result_test['lazyfree-lazy-eviction'] = ['no']
df_result_test['lazyfree-lazy-expire'] = ['no']
df_result_test['lazyfree-lazy-server-del'] = ['no']

# # default
# df_result_test['hash-max-ziplist-entries'] = [512]  #(256, 751)
# df_result_test['hash-max-ziplist-value'] = [64]  #(256, 751)
# df_result_test['activerehashing'] = ['yes']  #(256, 751)
# df_result_test['dynamic-hz'] = ['yes']  #(256, 751)
# df_result_test['hz'] = [10]  #(256, 751)
# df_result_test['save0_0'] = [900]
# df_result_test['save0_1'] = [1]
# df_result_test['save1_0'] = [300]
# df_result_test['save1_1'] = [10]
# df_result_test['save2_0'] = [60]
# df_result_test['save2_1'] = [10000]
# df_result_test['rdbcompression'] = ['yes']
# df_result_test['rdbchecksum'] = ['yes']
# df_result_test['rdb-save-incremental-fsync'] = ['yes']
# df_result_test['activedefrag'] = ['yes']
# df_result_test['active-defrag-threshold-lower'] = [10]
# df_result_test['active-defrag-threshold-upper'] = [100]
# df_result_test['active-defrag-cycle-min'] = [5]
# df_result_test['active-defrag-cycle-max'] = [75]
# df_result_test['maxmemory'] = [0]
# df_result_test['maxmemory-policy'] = 'noeviction'
# df_result_test['maxmemory-samples'] = [5]
# df_result_test['lazyfree-lazy-eviction'] = ['no']
# df_result_test['lazyfree-lazy-expire'] = ['no']
# df_result_test['lazyfree-lazy-server-del'] = ['no']

df_result_test2 = pd.DataFrame(df_result_test)
# for idx, ty in enumerate(list(df_result_test2.dtypes)):
#     if ty == 'O':
#         df_result_test2.iloc[0,idx].astype('category')
    
for elem in df_result_test2:
    if df_result_test2[elem].dtype == 'O':
        df_result_test2[elem] = df_result_test2[elem].astype('category')
        
lgbm_wrapper.predict(df_result_test2.iloc[:1,:])[0]
# df_result_test2.iloc[:1,:]

44791.058093691274

디폴트: 44428.73653257356

전체 BO 결과: 44791.058093691274

In [27]:
# df_result_test2

In [28]:
# lgbm_wrapper.predict(df_result_test2)

In [29]:
# # plot_importance()를 이영해 피처 중요도 시각화
# from lightgbm import plot_importance
# import matplotlib.pyplot as plt
# %matplotlib inline

# fig, ax = plt.subplots(figsize=(10, 12))
# plot_importance(lgbm_wrapper, ax= ax)

---

### 하이퍼파라미터 튜닝

In [30]:
# from sklearn.model_selection import GridSearchCV

# # 하이퍼파라미터 튜닝시 수행 속도를 향상시키기 위해 n_estimators를 200으로 감소
# lgbm_wrapper = LGBMRegressor(n_estimators = 200)

# params = {
#     'num_leaves':[32, 64],
#     'max_depth':[128, 160],
#     'min_child_samples':[60, 100],
#     'subsample':[0.8, 1]
# }

# # cv는 3으로 가정 (교차 검증을 자동으로 해주는건가?)
# gridcv = GridSearchCV(lgbm_wrapper, param_grid = params, cv=3)
# gridcv.fit(X_train, y_train, early_stopping_rounds = 30, eval_metric='auc', 
#           eval_set = [(X_train, y_train), (X_test, y_test)])

# print('GridSeaerchCV 최적 파라미터:', gridcv.best_params_)

In [31]:
# # 하이퍼 파라미터 적용 후 다시 생성
# lgbm_wrapper = LGBMRegressor(n_estimators = 1000,
#                             num_leaves = 32,
#                             subsample=0.8,
#                             min_child_samples=60,
#                             max_depth=128)

# evals = [(X_test, y_test)]

# lgbm_wrapper.fit(X_train, y_train, early_stopping_rounds=100,
#                 eval_metric='auc',
#                 eval_set=evals,
#                 verbose=True)

---

# BO

모델은 같은 것을 사용한다.

knob_default 딕셔너리 


1. 목적함수에서 지정한 범위knob 값을 랜덤하게 추출
2. 목적함수에는 init_knob 값이 지정되어있고 나머지 값들은 들어오는 값으로 초기화
목적함수 들어가기 전에 미리 선택을 해야 한다. 범위에 대해서 

cluster0 값만준다.
목적함수에서는 받은 cluster0을 할당한다
할당되지 않은 Knob에 대해서는 knob_init를 참고하여 값을 할당한다. 

결과값을 init_dict에 할당한다. 
목적함세어는 받은 cluster1을 할당한다.
할당되지 않은 Knob에 대해서는 knob_init을 참고한다. 




## 목적함수

In [94]:
noise_level = 0.1

def objective_func(knob_value, noise_level=noise_level):

    knob_dict = {}
    for knob in knobs_list: # 순서에 맞게 선언
        knob_dict[knob] = 'x'
    
#     print('###x 삽입###')
#     print(knob_dict)
    
    # knob_value 값 먼저 할당
    for idx,knob in enumerate(knobs):
        knob_dict[knob] = knob_value[idx]
    
#     print('##먼저할당##')
#     print(knob_dict)
    
    
    # 나머지 값 채우기 
    for knob ,knob_val in knob_dict.items():
        if knob_val == 'x':
            knob_dict[knob] = knobs_update[knob]
            
#     print('##나머지 update 반영 할당##')
#     print(knob_dict)
                    
    if 'activedefrag' in knob_dict:
        if knob_dict['activedefrag'] == 'no':  # activedefrag no인 경우 나머지 관련 knob들에 0할당!
            activ_var = ['active-defrag-threshold-lower', 'active-defrag-threshold-upper', 'active-defrag-cycle-min', 'active-defrag-cycle-max']
            for activ_knob in activ_var:
                knob_dict[activ_knob] = 0     
    
    knob_df = pd.DataFrame(knob_dict, index=[0])  # 데이터 프레임으로 변환
    
    # 카테고리 변수 변경
    for knob in category_knob:
        knob_df[knob] = knob_df[knob].astype('category') 
    
    
    return -lgbm_wrapper.predict(knob_df)[0]


## gp_minimize

In [33]:
%matplotlib inline
print(__doc__)

import numpy as np
np.random.seed(237)
import matplotlib.pyplot as plt
from skopt.plots import plot_gaussian_process
from skopt import gp_minimize

Automatically created module for IPython interactive environment


In [146]:
# knob 종류 및 바운드

# cluster_table = {
#     'cluster0':[
#         'save1_1', 
#         'lazyfree-lazy-eviction'
#     ],

#     'clsuter1':
#     [
#         'active-defrag-cycle-min',
#         'active-defrag-cycle-max',
#         'save0_0',
#         'save0_1',
#         'save2_0',
#         'save2_1',
#         'maxmemory-samples',
#         'hash-max-ziplist-entries',
#         'hash-max-ziplist-value',
#         'lazyfree-lazy-expire',
#         'dynamic-hz',
#         'hz'               
#                ],
# #     'other':[
# #         'activerehashing',
# #         'save1_0',
# #         'rdbcompression',
# #         'rdbchecksum',
# #         'rdb-save-incremental-fsync',
# #         'activedefrag',
# #         'active-defrag-threshold-lower',
# #         'active-defrag-threshold-upper',
# #         'maxmemory',
# #         'maxmemory-policy',
# #         'lazyfree-lazy-server-del'
# #     ]
# }

cluster_table = {

    'clsuter1':
    [
                'save1_1', 
        'lazyfree-lazy-eviction',

        'active-defrag-cycle-min',
        'active-defrag-cycle-max',
        'save0_0',
        'save0_1',
        'save2_0',
        'save2_1',
        'maxmemory-samples',
        'hash-max-ziplist-entries',
        'hash-max-ziplist-value',
        'lazyfree-lazy-expire',
        'dynamic-hz',
        'hz'               
               ],
    'other':[
        'activerehashing',
        'save1_0',
        'rdbcompression',
        'rdbchecksum',
        'rdb-save-incremental-fsync',
        'activedefrag',
        'active-defrag-threshold-lower',
        'active-defrag-threshold-upper',
        'maxmemory',
        'maxmemory-policy',
        'lazyfree-lazy-server-del'
     ]
}


knobs_update = {

    'hash-max-ziplist-entries': 512,
    'hash-max-ziplist-value': 64,
    'activerehashing': 'yes',
    'hz': 10,
    'dynamic-hz': 'yes',
    'save0_0': 900,
    'save0_1':1,
    'save1_0': 300,
    'save1_1':10,
    'save2_0': 60,
    'save2_1':10000,
    'rdbcompression': 'yes',
    'rdbchecksum': 'yes',
    'rdb-save-incremental-fsync': 'yes',
    'activedefrag': 'yes',
    'active-defrag-threshold-lower': 10,
    'active-defrag-threshold-upper': 100,
    'active-defrag-cycle-min': 5,
    'active-defrag-cycle-max': 75,
    'maxmemory': 0,
    'maxmemory-policy':'noeviction',
    'maxmemory-samples': 5,
    'lazyfree-lazy-eviction': 'no',
    'lazyfree-lazy-expire': 'no',
    'lazyfree-lazy-server-del': 'no'

}

# knobs_default = {

#     'hash-max-ziplist-entries': 270,
#     'hash-max-ziplist-value': 63,
#     'activerehashing': 'no',
#     'hz': 40,
#     'dynamic-hz': 'yes',
#     'save0_0': 829,
#     'save0_1':1,
#     'save1_0': 355,
#     'save1_1':100,
#     'save2_0': 31,
#     'save2_1':8478,
#     'rdbcompression': 'no',
#     'rdbchecksum': 'no',
#     'rdb-save-incremental-fsync': 'yes',
#     'activedefrag': 'yes',
#     'active-defrag-threshold-lower': 12,
#     'active-defrag-threshold-upper': 86,
#     'active-defrag-cycle-min': 10,
#     'active-defrag-cycle-max': 80,
#     'maxmemory': 2039,
#     'maxmemory-policy':'noeviction',
#     'maxmemory-samples': 3,
#     'lazyfree-lazy-eviction': 'no',
#     'lazyfree-lazy-expire': 'yes',
#     'lazyfree-lazy-server-del': 'no'

# }

SyntaxError: closing parenthesis '}' does not match opening parenthesis '[' on line 59 (<ipython-input-146-8c13de10c707>, line 72)

In [147]:
res_result = []

for i in range(5):
    print('###########{}##########'.format(i))
    
    iter_per_res = 0
    
    for cluster, knobs in cluster_table.items():
        print('#####{}#####'.format(cluster))
        gp_knobs_bound = []

        # 1. 클러스터의 bound 생성
#         print('{} bound 생성'.format(cluster))
        for knob in knobs: 
            gp_knobs_bound.append(knobs_rdb[knob])

#         print('{} gp 진행'.format(cluster))
        # 2. gp_minimize 진행
        res = gp_minimize(

                    objective_func,                  # 목적함수
                    gp_knobs_bound,      # bound
                    random_state=np.random.randint(1,1234),
#                     x0 = x0_list,
#                     y0 = y0_val, 
            
                    acq_func="EI",      
                    n_calls=50,      # 총 반복 횟수
                    n_random_starts=30,    # 초기 랜덤 값
                    noise=0.1**2,       
        )   

        # 결과 knob 값들
#         print('{} gp 완료'.format(cluster))
        gp_result = res.x
        iter_per_res = res.fun
        
#         x0_list = res.x
#         y0_val = res.fun
        
        for idx,knob in enumerate(knobs):
            knobs_update[knob] = gp_result[idx]
        
        print('{}-th'.format(i))
        print(res.func_vals)
        print(res.fun)
        
    print(iter_per_res)
    print()
        


###########0##########
#####clsuter1#####
0-th
[-44428.73653257 -44428.73653257 -44428.73653257 -44618.39021575
 -44618.39021575 -44618.39021575 -44249.19899715 -44249.19899715
 -44618.39021575 -44428.73653257 -44428.73653257 -44618.39021575
 -44428.73653257 -44597.61136068 -44597.61136068 -44618.39021575
 -44350.76420385 -44597.61136068 -44597.61136068 -44597.61136068
 -44428.73653257 -44618.39021575 -44350.76420385 -44350.76420385
 -44350.76420385 -44618.39021575 -44618.39021575 -44618.39021575
 -44618.39021575 -44618.39021575 -44618.39021575 -44597.61136068
 -44618.39021575 -44618.39021575 -44618.39021575 -44618.39021575
 -44597.61136068 -44618.39021575 -44618.39021575 -44618.39021575
 -44618.39021575 -44618.39021575 -44618.39021575 -44597.61136068
 -44597.61136068 -44618.39021575 -44618.39021575 -44618.39021575
 -44618.39021575 -44618.39021575]
-44618.390215749234
-44618.390215749234

###########1##########
#####clsuter1#####
1-th
[-44428.73653257 -44249.19899715 -44428.73653257 -4

In [148]:
knobs_update

{'hash-max-ziplist-entries': 657,
 'hash-max-ziplist-value': 186,
 'activerehashing': 'yes',
 'hz': 2,
 'dynamic-hz': 'no',
 'save0_0': 773,
 'save0_1': 3,
 'save1_0': 300,
 'save1_1': 19,
 'save2_0': 87,
 'save2_1': 8385,
 'rdbcompression': 'yes',
 'rdbchecksum': 'yes',
 'rdb-save-incremental-fsync': 'yes',
 'activedefrag': 'yes',
 'active-defrag-threshold-lower': 10,
 'active-defrag-threshold-upper': 100,
 'active-defrag-cycle-min': 13,
 'active-defrag-cycle-max': 83,
 'maxmemory': 0,
 'maxmemory-policy': 'noeviction',
 'maxmemory-samples': 5,
 'lazyfree-lazy-eviction': 'yes',
 'lazyfree-lazy-expire': 'no',
 'lazyfree-lazy-server-del': 'no'}

In [151]:
# 전체 데이터 대상
res_result = []

cluster_table_T = {

    'clsuter1':
    [
        'save1_1', 
        'lazyfree-lazy-eviction',

        'active-defrag-cycle-min',
        'active-defrag-cycle-max',
        'save0_0',
        'save0_1',
        'save2_0',
        'save2_1',
        'maxmemory-samples',
        'hash-max-ziplist-entries',
        'hash-max-ziplist-value',
        'lazyfree-lazy-expire',
        'dynamic-hz',
        'hz',
        'activerehashing',
        'save1_0',
        'rdbcompression',
        'rdbchecksum',
        'rdb-save-incremental-fsync',
        'activedefrag',
        'active-defrag-threshold-lower',
        'active-defrag-threshold-upper',
        'maxmemory',
        'maxmemory-policy',
        'lazyfree-lazy-server-del'

               ]
}

knobs_update = {

    'hash-max-ziplist-entries': 512,
    'hash-max-ziplist-value': 64,
    'activerehashing': 'yes',
    'hz': 10,
    'dynamic-hz': 'yes',
    'save0_0': 900,
    'save0_1':1,
    'save1_0': 300,
    'save1_1':10,
    'save2_0': 60,
    'save2_1':10000,
    'rdbcompression': 'yes',
    'rdbchecksum': 'yes',
    'rdb-save-incremental-fsync': 'yes',
    'activedefrag': 'yes',
    'active-defrag-threshold-lower': 10,
    'active-defrag-threshold-upper': 100,
    'active-defrag-cycle-min': 5,
    'active-defrag-cycle-max': 75,
    'maxmemory': 0,
    'maxmemory-policy':'noeviction',
    'maxmemory-samples': 5,
    'lazyfree-lazy-eviction': 'no',
    'lazyfree-lazy-expire': 'no',
    'lazyfree-lazy-server-del': 'no'

}


# knobs_update = {

#     'hash-max-ziplist-entries': 270,
#     'hash-max-ziplist-value': 63,
#     'activerehashing': 'no',
#     'hz': 40,
#     'dynamic-hz': 'yes',
#     'save0_0': 829,
#     'save0_1':1,
#     'save1_0': 355,
#     'save1_1':100,
#     'save2_0': 31,
#     'save2_1':8478,
#     'rdbcompression': 'no',
#     'rdbchecksum': 'no',
#     'rdb-save-incremental-fsync': 'yes',
#     'activedefrag': 'yes',
#     'active-defrag-threshold-lower': 12,
#     'active-defrag-threshold-upper': 86,
#     'active-defrag-cycle-min': 10,
#     'active-defrag-cycle-max': 80,
#     'maxmemory': 2039,
#     'maxmemory-policy':'noeviction',
#     'maxmemory-samples': 3,
#     'lazyfree-lazy-eviction': 'no',
#     'lazyfree-lazy-expire': 'yes',
#     'lazyfree-lazy-server-del': 'no'

# }



for i in range(2):
    print('###########{}##########'.format(i))
    
    iter_per_res = 0
    
    for cluster, knobs in cluster_table_T.items():
        print('#####{}#####'.format(cluster))
        gp_knobs_bound = []

        # 1. 클러스터의 bound 생성
#         print('{} bound 생성'.format(cluster))
        for knob in knobs: 
            gp_knobs_bound.append(knobs_rdb[knob])

#         print('{} gp 진행'.format(cluster))
        # 2. gp_minimize 진행
        res = gp_minimize(

                    objective_func,                  # 목적함수
                    gp_knobs_bound,      # bound
                    random_state=np.random.randint(1,1234),
#                     x0 = x0_list,
#                     y0 = y0_val, 
            
                    acq_func="EI",      
                    n_calls=30,      # 총 반복 횟수
                    n_random_starts=5,    # 초기 랜덤 값
                    noise=0.1**2,       
        )   

        # 결과 knob 값들
#         print('{} gp 완료'.format(cluster))
        gp_result = res.x
        iter_per_res = res.fun
        
#         x0_list = res.x
#         y0_val = res.fun
        
        for idx,knob in enumerate(knobs):
            knobs_update[knob] = gp_result[idx]
        
        print('{}-th'.format(i))
        print(res.func_vals)
        print(res.fun)
        
    print(iter_per_res)
    print()
        


###########0##########
#####clsuter1#####
0-th
[-44618.39021575 -44464.81720365 -44428.73653257 -44618.39021575
 -44339.93518275 -44597.61136068 -44618.39021575 -44618.39021575
 -44350.76420385 -44618.39021575 -44618.39021575 -44597.61136068
 -44249.29409944 -44597.61136068 -44618.39021575 -44618.39021575
 -44618.39021575 -44372.84884351 -44618.39021575 -44597.61136068
 -44618.39021575 -44332.57532435 -44597.61136068 -44597.61136068
 -44481.77898713 -44618.39021575 -44618.39021575 -44618.39021575
 -44597.61136068 -44618.39021575]
-44618.390215749234
-44618.390215749234

###########1##########
#####clsuter1#####
1-th
[-44481.77898713 -44597.61136068 -44343.78622585 -44597.61136068
 -44358.50862879 -44380.94917837 -44618.39021575 -44350.76420385
 -44618.39021575 -44618.39021575 -44618.39021575 -44618.39021575
 -44350.76420385 -44339.10816538 -44597.61136068 -44618.39021575
 -44618.39021575 -44618.39021575 -44618.39021575 -44618.39021575
 -44618.39021575 -44618.39021575 -44597.61136068 -4

In [152]:
knobs_update

{'hash-max-ziplist-entries': 561,
 'hash-max-ziplist-value': 179,
 'activerehashing': 'yes',
 'hz': 18,
 'dynamic-hz': 'no',
 'save0_0': 782,
 'save0_1': 7,
 'save1_0': 257,
 'save1_1': 91,
 'save2_0': 32,
 'save2_1': 10742,
 'rdbcompression': 'no',
 'rdbchecksum': 'no',
 'rdb-save-incremental-fsync': 'no',
 'activedefrag': 'yes',
 'active-defrag-threshold-lower': 21,
 'active-defrag-threshold-upper': 97,
 'active-defrag-cycle-min': 17,
 'active-defrag-cycle-max': 86,
 'maxmemory': 1791,
 'maxmemory-policy': 'allkeys-lru',
 'maxmemory-samples': 5,
 'lazyfree-lazy-eviction': 'no',
 'lazyfree-lazy-expire': 'no',
 'lazyfree-lazy-server-del': 'no'}

In [51]:
li = []
for value in knobs_update.values():
    li.append(value)

In [52]:
li

[578,
 30,
 'yes',
 1,
 'yes',
 878,
 2,
 347,
 82,
 58,
 9335,
 'yes',
 'no',
 'no',
 'yes',
 13,
 71,
 24,
 83,
 2850,
 'allkeys-random',
 4,
 'no',
 'yes',
 'no']

---

In [181]:
# gp 결과 정리 함수
def li_to_str(result_list):
    for idx,knob in enumerate(list(knobs_rdb.keys())):  # knobs_rdb 키마다 접근
        print("{} ".format(knob), end='')
        print('{}'.format(result_list[idx]))

In [74]:
# gp 결과
li_to_str(res.x)

hash-max-ziplist-entries 305
save0_0 722
save0_1 5
save1_1 12
save2_1 9163
rdb-save-incremental-fsync no
lazyfree-lazy-expire yes


## gp 결과 정리

### gp 분류된것
"hash-max-ziplist-entries" : 284
'hash-max-ziplist-value' 22
'activerehashing': 'yes'
'hz' 32
'dynamic-hz' : 'yes'
'save0_0' : 877
'save0_1'2
'save1_0'272
'save1_1' : 12
'save2_0'33
'save2_1' : 11669 
'rdbcompression'no
'rdbchecksum' yes
"rdb-save-incremental-fsync" 'no'
'active-defrag-threshold-lower'3
'active-defrag-threshold-upper': 87
'active-defrag-cycle-min' : 25
'active-defrag-cycle-max' : 80
'maxmemory'1298
maxmemory-policy : 'allkeys-random' 
'maxmemory-samples'6
"lazyfree-lazy-eviction"  'no'
'lazyfree-lazy-expire' yes
'lazyfree-lazy-server-del' yes


### gp 분류된것2
hash-max-ziplist-entries 502
hash-max-ziplist-value 213
activerehashing yes
hz 32
dynamic-hz yes
save0_0 760
save0_1 3
save1_0 426
save1_1 24
save2_0 37
save2_1 8052 
rdbcompression no
rdbchecksum' no
rdb-save-incremental-fsync yes
active-defrag-threshold-lower 4
active-defrag-threshold-upper 72
active-defrag-cycle-min 14
active-defrag-cycle-max 70
maxmemory 1552
maxmemory-policy allkeys-random
maxmemory-samples 6
lazyfree-lazy-eviction yes
lazyfree-lazy-expire yes
lazyfree-lazy-server-del no
[575,
 94,
 'no',
 10,
 'yes',
 1072,
 5,
 465,
 24,
 83,
 9398,
 'yes',
 'yes',
 'no',
 26,
 81,
 20,
 75,
 1539,
 'allkeys-lfu',
 4,
 'yes',
 'no',
 'no']

### gp_T
hash-max-ziplist-entries 575
hash-max-ziplist-value 94
activerehashing no
hz 10
dynamic-hz yes
save0_0 1072
save0_1 5
save1_0 465
save1_1 24
save2_0 83
save2_1 9398
rdbcompression yes
rdbchecksum yes
rdb-save-incremental-fsync no
active-defrag-threshold-lower 26
active-defrag-threshold-upper 81
active-defrag-cycle-min 20
active-defrag-cycle-max 75
maxmemory 1539
maxmemory-policy allkeys-lfu
maxmemory-samples 4
lazyfree-lazy-eviction yes
lazyfree-lazy-expire no
lazyfree-lazy-server-del no

